## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,2022-08-11 02:54:07,-42.8794,147.3294,57.25,71,0,5.75,clear sky
1,1,Tasiilaq,GL,2022-08-11 02:53:16,65.6145,-37.6368,33.94,77,89,3.13,overcast clouds
2,2,Torbay,CA,2022-08-11 02:57:21,47.6666,-52.7314,58.26,89,100,5.99,overcast clouds
3,3,Hasaki,JP,2022-08-11 02:56:52,35.7333,140.8333,93.29,79,30,21.88,moderate rain
4,4,Souillac,MU,2022-08-11 02:57:22,-20.5167,57.5167,66.58,72,40,13.80,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 68
What is the maximum temperature you would like for your trip? 82


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Arraial Do Cabo,BR,2022-08-11 02:57:22,-22.9661,-42.0278,68.09,67,75,19.75,broken clouds
7,7,Cherskiy,RU,2022-08-11 02:57:22,68.7500,161.3000,69.80,51,10,2.57,clear sky
10,10,Nkoteng,CM,2022-08-11 02:57:23,4.5167,12.0333,68.27,99,100,4.81,light rain
11,11,Iracoubo,GF,2022-08-11 02:57:23,5.4802,-53.2011,77.61,88,44,8.68,scattered clouds
16,16,Vaini,TO,2022-08-11 02:57:25,-21.2000,-175.2000,78.96,94,75,14.97,broken clouds
17,17,Sao Filipe,CV,2022-08-11 02:57:25,14.8961,-24.4956,77.58,87,62,8.63,broken clouds
18,18,Bethel,US,2022-08-11 02:54:56,41.3712,-73.4140,75.07,76,2,3.65,clear sky
19,19,Kipini,KE,2022-08-11 02:57:26,-2.5257,40.5262,73.98,85,48,16.62,light rain
21,21,Chokurdakh,RU,2022-08-11 02:57:26,70.6333,147.9167,72.10,42,100,9.46,overcast clouds
23,23,San Quintin,MX,2022-08-11 02:55:57,30.4833,-115.9500,73.08,78,94,9.28,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Arraial Do Cabo,BR,68.09,broken clouds,-22.9661,-42.0278,
7,Cherskiy,RU,69.80,clear sky,68.7500,161.3000,
10,Nkoteng,CM,68.27,light rain,4.5167,12.0333,
11,Iracoubo,GF,77.61,scattered clouds,5.4802,-53.2011,
16,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
17,Sao Filipe,CV,77.58,broken clouds,14.8961,-24.4956,
18,Bethel,US,75.07,clear sky,41.3712,-73.4140,
19,Kipini,KE,73.98,light rain,-2.5257,40.5262,
21,Chokurdakh,RU,72.10,overcast clouds,70.6333,147.9167,
23,San Quintin,MX,73.08,overcast clouds,30.4833,-115.9500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping city.")
        

Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not found... skipping city.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Arraial Do Cabo,BR,68.09,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
7,Cherskiy,RU,69.80,clear sky,68.7500,161.3000,Gostinitsa
10,Nkoteng,CM,68.27,light rain,4.5167,12.0333,Résidence OOCLO-Cannelle
11,Iracoubo,GF,77.61,scattered clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
16,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))